## 활성화 함수 및 손실 함수 정의

In [ ]:
import numpy as np

# 활성화 함수 및 미분
def sigmoid(x): # 출력층
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def relu(x): # 은닉층
    return np.maximum(0, x)
def relu_derivative(x):
    return (x > 0).astype(float)

# 손실 함수 (MSE) 및 미분
def mse_loss(y_hat, y):
    return 0.5 * np.mean((y_hat - y)2)
def mse_loss_derivative(y_hat, y):
    return (y_hat - y) / y.size

## 신경망 클래스: 초기화 및 순전파

In [ ]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        # 가중치 및 편향 초기화
        self.W1 = np.random.randn(hidden_size, input_size) * 0.01
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size) * 0.01
        self.b2 = np.zeros((output_size, 1))

    def forward(self, x):
        # 순전파 계산
        self.z1 = np.dot(self.W1, x) + self.b1 # 은닉층 선형 변환
        self.a1 = relu(self.z1) # 은닉층 활성화
        self.z2 = np.dot(self.W2, self.a1) + self.b2 # 출력층 선형 변환
        self.y_hat = sigmoid(self.z2) # 출력층 활성화 (예측값)
        return self.y_hat
    
    def backward(self, x, y, y_hat):
        # 역전파 계산
        m = y.size # 데이터 샘플 수

        # 1. 출력층 오차 항수 (delta2) 계산
        delta2 = mse_loss_derivative(y_hat, y) * sigmoid_derivative(self.z2)

        # 2. 은닉층 오차 항수 (delta1) 계산 (역전파)
        # delta1 = (W2.T * delta2) element-wise * relu_derivative(z1)
        delta1 = np.dot(self.W2.T, delta2) * relu_derivative(self.z1)

        # 3. 각 가중치 및 편향 기울기 계산
        dW2 = np.dot(delta2, self.a1.T) / m
        db2 = np.sum(delta2, axis=1, keepdims=True) / m
        dW1 = np.dot(delta1, x.T) / m
        db1 = np.sum(delta1, axis=1, keepdims=True) / m

        return dW1, db1, dW2, db2
    
    def update_params(self, dW1, db1, dW2, db2, learning_rate):
        # 가중치 및 편향 업데이트 (경사하강법)
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def train(self, X, Y, learning_rate, epochs):
        # 지정된 epoch 수만큼 학습 반복
        for epoch in range(epochs):
            y_hat = self.forward(X) # 순전파
            dW1, db1, dW2, db2 = self.backward(X, Y, y_hat) # 역전파
            self.update_params(dW1, db1, dW2, db2, learning_rate) # 업데이트

            loss = mse_loss(y_hat, Y) # 손실 계산
            if epoch % 100 == 0: # 100 epoch마다 손실 출력
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

## 학습 데이터 및 실행

In [ ]:
# 학습 데이터 생성 (간단한 AND 게이트)
# X: 2 features, 4 samples
# Y: 1 output, 4 samples
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
Y = np.array([[0, 0, 0, 1]])

# 모델 생성 및 학습
net = TwoLayerNet(input_size=2, hidden_size=3, output_size=1)
net.train(X, Y, learning_rate=0.1, epochs=1000)

# 학습 후 예측
test_x1 = np.array([[1], [1]])
prediction1 = net.forward(test_x1)
print(f"Prediction for [1, 1]: {prediction1[0][0]:.4f}") # Expected: ~1

test_x2 = np.array([[0], [1]])
prediction2 = net.forward(test_x2)
print(f"Prediction for [0, 1]: {prediction2[0][0]:.4f}") # Expected: ~0

## Keras로 만들어보기

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# 1. 학습 데이터 생성 (AND 게이트)
# X: 입력. 각 행이 하나의 샘플. (4 samples, 2 features)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
# Y: 출력. (4 samples, 1 output)
Y = np.array([[0], [0], [0], [1]])

# 2. 모델 구성
# 입력 특징 2개, 은닉층 뉴런 3개 (ReLU 활성화), 출력 뉴런 1개 (Sigmoid 활성화)
model = Sequential([
    Dense(3, activation='relu', input_shape=(2,)), # 은닉층
    Dense(1, activation='sigmoid') # 출력층
])

# 3. 모델 컴파일
# 옵티마이저: SGD (Stochastic Gradient Descent), 손실 함수: MSE (Mean Squared Error)
# 학습률(learning_rate)은 SGD 옵티마이저 내에서 설정
optimizer = SGD(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy']) # mse도 가능

# 모델 요약 정보 출력
model.summary()

# 4. 모델 학습
print("\nTraining with Keras...")
# epochs를 1000으로 설정 (NumPy 예제와 유사하게)
# verbose=0으로 설정하여 학습 중 로그 출력을 최소화할 수 있음
# verbose=1이면 진행 막대 표시, verbose=2이면 에포크당 한 줄
history = model.fit(X, Y, epochs=1000, verbose=0, batch_size=4) # 전체 데이터를 하나의 배치로 사용
print("Training finished.")

# 학습 마지막 손실 값 출력
final_loss = history.history['loss'][-1]
print(f"Final loss: {final_loss:.4f}")


# 5. 학습 후 예측
print("\nPredictions after training (Keras):")
predictions = model.predict(X)
for i in range(X.shape[0]):
    print(f"Input: {X[i]}, Prediction: {predictions[i][0]:.4f} (Expected: {Y[i][0]})")

test_x1 = np.array([[1, 1]])
prediction1 = model.predict(test_x1)
print(f"Prediction for [1, 1]: {prediction1[0][0]:.4f} (Expected: ~1)")

test_x2 = np.array([[0, 1]])
prediction2 = model.predict(test_x2)
print(f"Prediction for [0, 1]: {prediction2[0][0]:.4f} (Expected: ~0)")

## PyTorch로 만들어보기

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 0. 재현성을 위한 시드 설정 (선택 사항)
# torch.manual_seed(0)
# np.random.seed(0)

# 1. 학습 데이터 생성 (AND 게이트) 및 Tensor 변환
# X: 입력. 각 행이 하나의 샘플. (4 samples, 2 features)
X_numpy = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
# Y: 출력. (4 samples, 1 output)
Y_numpy = np.array([[0], [0], [0], [1]], dtype=np.float32)

X_torch = torch.tensor(X_numpy)
Y_torch = torch.tensor(Y_numpy)

# 2. 신경망 모델 정의
# 입력 특징 2개, 은닉층 뉴런 3개 (ReLU 활성화), 출력 뉴런 1개 (Sigmoid 활성화)
class TwoLayerNetPyTorch(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNetPyTorch, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 입력층 -> 은닉층
        self.relu = nn.ReLU()                         # ReLU 활성화
        self.fc2 = nn.Linear(hidden_size, output_size) # 은닉층 -> 출력층
        self.sigmoid = nn.Sigmoid()                   # Sigmoid 활성화

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

input_size = 2
hidden_size = 3
output_size = 1
model_pytorch = TwoLayerNetPyTorch(input_size, hidden_size, output_size)

# 모델 구조 출력
print(model_pytorch)

# 3. 손실 함수 및 옵티마이저 정의
# 손실 함수: MSE (Mean Squared Error)
criterion = nn.MSELoss()
# 옵티마이저: SGD (Stochastic Gradient Descent)
learning_rate = 0.1
optimizer_pytorch = optim.SGD(model_pytorch.parameters(), lr=learning_rate)

# 4. 모델 학습
print("\nTraining with PyTorch...")
epochs = 1000
for epoch in range(epochs):
    # 순전파
    outputs = model_pytorch(X_torch)
    loss = criterion(outputs, Y_torch)

    # 역전파 및 가중치 업데이트
    optimizer_pytorch.zero_grad() # 이전 에포크의 기울기 초기화
    loss.backward()             # 역전파 수행 (기울기 계산)
    optimizer_pytorch.step()      # 가중치 업데이트

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
print("Training finished.")
final_loss_pytorch = loss.item()
print(f"Final loss: {final_loss_pytorch:.4f}")

# 5. 학습 후 예측
print("\nPredictions after training (PyTorch):")
model_pytorch.eval() # 평가 모드로 전환 (Dropout, BatchNorm 등에 영향)
with torch.no_grad(): # 기울기 계산 비활성화
    predictions_pytorch = model_pytorch(X_torch)
    for i in range(X_torch.shape[0]):
        print(f"Input: {X_numpy[i]}, Prediction: {predictions_pytorch[i].item():.4f} (Expected: {Y_numpy[i][0]})")

    test_x1_torch = torch.tensor([[1, 1]], dtype=torch.float32)
    prediction1_torch = model_pytorch(test_x1_torch)
    print(f"Prediction for [1, 1]: {prediction1_torch.item():.4f} (Expected: ~1)")

    test_x2_torch = torch.tensor([[0, 1]], dtype=torch.float32)
    prediction2_torch = model_pytorch(test_x2_torch)
    print(f"Prediction for [0, 1]: {prediction2_torch.item():.4f} (Expected: ~0)")
